In [9]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, concatenate_datasets, load_from_disk
import time
import pdb
import glob

In [5]:
chunks = glob.glob('chunks/featurized_chunks_*')


In [10]:
datasets = []
for c in chunks:
    print(f"Reading chunk {c}")
    dataset = load_from_disk(c)
    datasets.append(dataset)

# Concatenate all datasets into a single dataset
dataset = concatenate_datasets(datasets[:2])

Reading chunk chunks/featurized_chunks_603
Reading chunk chunks/featurized_chunks_258
Reading chunk chunks/featurized_chunks_294
Reading chunk chunks/featurized_chunks_851
Reading chunk chunks/featurized_chunks_409
Reading chunk chunks/featurized_chunks_455
Reading chunk chunks/featurized_chunks_268
Reading chunk chunks/featurized_chunks_75
Reading chunk chunks/featurized_chunks_574
Reading chunk chunks/featurized_chunks_10
Reading chunk chunks/featurized_chunks_315
Reading chunk chunks/featurized_chunks_722
Reading chunk chunks/featurized_chunks_510
Reading chunk chunks/featurized_chunks_193
Reading chunk chunks/featurized_chunks_595
Reading chunk chunks/featurized_chunks_753
Reading chunk chunks/featurized_chunks_94
Reading chunk chunks/featurized_chunks_95
Reading chunk chunks/featurized_chunks_594
Reading chunk chunks/featurized_chunks_8
Reading chunk chunks/featurized_chunks_244
Reading chunk chunks/featurized_chunks_415
Reading chunk chunks/featurized_chunks_287
Reading chunk chu

In [11]:
dataset

Dataset({
    features: ['user_id', 'rating', 'helpful_vote', 'timestamp', 'asin', 'text', 'title', 'parent_asin', 'verified_purchase', 'embedding'],
    num_rows: 100000
})

In [12]:
# Reshard dataset to 32 shards
dataset.save_to_disk("dataset", num_shards=32)

Saving the dataset (0/32 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [13]:
# Now reload
dataset = load_from_disk("dataset",keep_in_memory=True)

Loading dataset from disk:   0%|          | 0/32 [00:00<?, ?it/s]

In [14]:
dataset

Dataset({
    features: ['user_id', 'rating', 'helpful_vote', 'timestamp', 'asin', 'text', 'title', 'parent_asin', 'verified_purchase', 'embedding'],
    num_rows: 100000
})

In [15]:
train_test_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_dataset['train']
test_dataset = train_test_dataset['test']
# borrow another 10% from the train set to act as validation
train_valid= train_dataset.train_test_split(test_size=0.1)
train_dataset = train_valid['train']
valid_dataset = train_valid['test']


In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import sklearn

# Total size of data
N = len(train_dataset)

X = [[1] + fv['embedding'] for fv in train_dataset]
y = train_dataset['rating']

X_valid = [[1] + fv['embedding'] for fv in valid_dataset]
y_valid = valid_dataset['rating']

X_test = [[1] + fv['embedding'] for fv in test_dataset]
y_test = test_dataset['rating']

model_lr = sklearn.linear_model.Ridge(1, fit_intercept=False)
model_lr.fit(X_train, y_train)

y_pred = model_lr.predict(X_valid)
mse = mean_squared_error(y_valid, y_pred)
print(f"Mean Squared Error: {mse}") 